In [20]:
import math

def bootstrap_from_swaps(tenors, par_rates, accruals, seed_D1=None):
    """
    tenors   : [t1, t2, ...] in years (include each coupon date)
    par_rates: [S_k for each maturity t_k], same length as tenors
              (e.g., if coupons are semiannual, S appears at pillars 2,4,6,...)
    accruals : [alpha_i] per period (e.g., 0.5 for semiannual)
    seed_D1  : optional known D(t1); otherwise set from a money-market instrument
    """
    n = len(tenors)
    D = [None]*n
    # If you have a deposit or FRA for the first period, set D[0] from that.
    if seed_D1 is not None:
        D[0] = seed_D1
    else:
        # crude seed: use exp(-S1 * t1) if no short instrument provided
        D[0] = math.exp(-par_rates[0] * tenors[0])

    # Iterate pillars
    for k in range(1, n):
        S = par_rates[k]
        alpha_k = accruals[k]
        fixed_sum_known = sum(accruals[i] * D[i] for i in range(k))

        def f(Dk):
            return S * (fixed_sum_known + alpha_k * Dk) - (1.0 - Dk)

        fprime = S * alpha_k + 1.0
        # initial guess for Dk
        Dk = math.exp(-S * tenors[k])

        for _ in range(12):
            step = f(Dk) / fprime
            Dk -= step
            if abs(step) < 1e-12:
                break

        D[k] = Dk

    return D

# Example (semiannual for 2Y): t=[0.5,1.0,1.5,2.0], S=[3.0%,3.0%,3.2%,3.4%]
tenors    = [1, 2, 3, 4, 5, 6, 7]             # years
alphas    = [1.0]*7                           # annual accrual
par = [0.0364, 0.0358, 0.0362, 0.0368, 0.0376, 0.0385, 0.0394]  # decimals
discounts = bootstrap_from_swaps(tenors, par, alphas, seed_D1=0.9850)
print(discounts)

[0.985, 0.931393126086117, 0.8981148126188792, 0.8646085145212733, 0.8304406528129262, 0.7957458367043726, 0.7609881316681772]
